# Objectif

Dans ce code, des données sont extraites de pages web pour chaque entreprise listée dans `liste_url`, en utilisant les bibliothèques **requests**, **BeautifulSoup (bs)** et **pandas**. Le code a été mis à jour pour inclure une sélection aléatoire de l'en-tête User-Agent, ce qui peut aider à prévenir le blocage par certains sites web qui limitent les accès automatisés.

1. **Initialisation d'une liste pour stocker les données**  `donnees_entreprises = []`
	
2. **Boucle sur chaque URL** dans `liste_url` :
	- **Sélection aléatoire d'un User-Agent** : Un navigateur est choisi aléatoirement parmi une liste prédéfinie (`csts.BROWSERS`). Cela permet de varier l'en-tête User-Agent envoyé avec chaque requête, ce qui peut éviter de se faire bloquer par les mesures anti-scraping des sites web.
	- **Envoi d'une requête HTTP GET** : Une requête est envoyée à l'URL avec l'en-tête User-Agent spécifié.
	- **Vérification du statut de la réponse** : Le code vérifie que la réponse a un statut HTTP 200, qui indique une réponse réussie.
	- **Extraction du contenu HTML** : Le contenu HTML de la réponse est stocké.

3. **Analyse et extraction des données avec BeautifulSoup**
	-	**Analyse du contenu HTML :** Le contenu HTML est analysé à l'aide de BeautifulSoup.
	- **Extraction des informations spécifiques** : Le code extrait le nom de l'entreprise, l'adresse, et les numéros SIREN, SIRET et TVA à partir des éléments HTML spécifiés.

4. **Création d'un dataframe pour chaque entreprise** : Les informations extraites sont stockées dans un dataframe pandas.
	
5.  **Ajout du dataframe à la liste** : Le dataframe créé pour chaque entreprise est ajouté à la liste `donnees_entreprises`.

7. **Concaténation des dataframes en un seul** : Après avoir traité toutes les URLs, tous les dataframes sont combinés en un seul dataframe. Cela est fait en utilisant `pd.concat`, qui concatène les dataframes listés, avec `axis=0` pour concaténer verticalement et `ignore_index=True` pour réinitialiser les indices dans le dataframe final.

# Importation des packages & modules nécessaires

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
import time

import random
import yaml

import os
import sys

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

import src.constants as csts
import src.paths as paths
from src.utils import get_current_date

In [2]:
# Chargement des headers
with open("headers.yml") as f_headers:
    browser_headers = yaml.safe_load(f_headers)

# Test avec 2 URLs

A l'avenir, ces listes seront remplacées par un fichier excel contenant 2 colonnes : une colonne "Nom" et une colonne "SIREN"

In [3]:
# Nom "brutes"
liste_nom = ["France Croco", "Wahaus"]
print(">> Liste de noms avant préparation :", liste_nom)

# Noms formatés pour les injecter dans l'URL
liste_nom = [nom.strip().lower().replace(" ", "-") for nom in liste_nom]
print(">> Liste de noms après préparation :", liste_nom)

# Numéros SIREN
liste_numero_siren = ["501737761", "537443228"]
print(">> Numéros SIREN :", liste_numero_siren)

# Création de la liste des urls
liste_url = [csts.BASE_URL + nom + "-" + siren + csts.HTML for nom, siren in list(zip(liste_nom, liste_numero_siren))]
print(liste_url)

>> Liste de noms avant préparation : ['France Croco', 'Wahaus']
>> Liste de noms après préparation : ['france-croco', 'wahaus']
>> Numéros SIREN : ['501737761', '537443228']
['https://www.societe.com/societe/france-croco-501737761.html', 'https://www.societe.com/societe/wahaus-537443228.html']


# Lancement du scraping

In [4]:
# Création d'une liste vide pour stocker les données de chaque entreprise
donnees_entreprises = []

print("Début de l'extraction des données des entreprises.")

# Boucle sur chaque URL dans la liste `liste_url`
for url in liste_url:
    try:
        print(f" >> Traitement de l'URL : {url}")

        # Choix aléatoire d'un nom de navigateur dans la liste des navigateurs disponibles
        browser_name = random.choice(csts.BROWSERS)
        print(f"    ** Navigateur choisi : {browser_name}")

        # Récupération des headers correspondants au navigateur choisi
        headers = browser_headers[browser_name]

        # Envoi d'une requête HTTP GET à l'URL avec un User-Agent spécifié
        response = requests.get(url=url, headers=headers)

        # Met le programme en pause pendant 10 secondes.
        print("    ** Pause de 10 secondes...")
        time.sleep(csts.TIME_SLEEP)

        # Obtention du contenu HTML de la réponse
        html = response.content

        # Analyse du contenu HTML avec BeautifulSoup
        contenu_page = bs(html, "html.parser")

        # Extraction de la première occurrence d'une table avec les classes spécifiées
        contenu_page = contenu_page.find_all(
            name="table",
            class_="Table identity mt-16",
        )[0]

        # Extraction du nom de l'entreprise
        nom = contenu_page.find(name="td", class_="break-word").text
        print(f"    ** Nom de l'entreprise extrait : {nom}")

        # Extraction de l'adresse
        adresse = contenu_page.find(name="a", class_="Lien secondaire").text.strip()
        print(f"    ** Adresse extraite : {adresse}")

        # Extraction du numéro SIREN
        numero_siren = contenu_page.find(name="div", id="siren_number").text
        print(f"    ** Numéro SIREN extrait : {numero_siren}")

        # Extraction du numéro SIRET
        numero_siret = contenu_page.find(name="div", id="siret_number").text
        print(f"    ** Numéro SIRET extrait : {numero_siret}")

        # Extraction du numéro TVA
        numero_tva = contenu_page.find(name="div", id="tva_number").text
        print(f"    ** Numéro TVA extrait : {numero_tva}")

        # Création d'un dataframe avec les données extraites
        donnees_entreprise = pd.DataFrame(
            [
                {
                    "Nom Commerciaux": nom,
                    "Adresse": adresse,
                    "Numéro SIREN": numero_siren,
                    "Numéro SIRET": numero_siret,
                    "Numéro TVA": numero_tva,
                }
            ]
        )

        # Ajout du dataframe à la liste des dataframes des entreprises
        donnees_entreprises.append(donnees_entreprise)
    except Exception as e:
        print(e)
        continue

# Concaténation de tous les dataframes en un seul
donnees_entreprises = pd.concat(objs=donnees_entreprises, axis=0, ignore_index=True)
print("Toutes les données ont été extraites et concaténées dans un dataframe unique.")

Début de l'extraction des données des entreprises.
 >> Traitement de l'URL : https://www.societe.com/societe/france-croco-501737761.html
    ** Navigateur choisi : Chrome
    ** Pause de 10 secondes...
    ** Nom de l'entreprise extrait : FRANCE CROCO
    ** Adresse extraite : ZA LA MARE AUX RAINES NORD 50190 PERIERS
    ** Numéro SIREN extrait : 501737761
    ** Numéro SIRET extrait : 50173776100020
    ** Numéro TVA extrait : FR81501737761
 >> Traitement de l'URL : https://www.societe.com/societe/wahaus-537443228.html
    ** Navigateur choisi : Firefox
    ** Pause de 10 secondes...
    ** Nom de l'entreprise extrait : WAHAUS
    ** Adresse extraite : 31 AV D EYLAU 75116 PARIS
    ** Numéro SIREN extrait : 537443228
    ** Numéro SIRET extrait : 53744322800011
    ** Numéro TVA extrait : FR61537443228
Toutes les données ont été extraites et concaténées dans un dataframe unique.


# Export au format .XLSX

In [5]:
file_name = get_current_date() + paths.EXCEL_FILE_NAME
file_path = paths.DATA_DIR_PATH / file_name
donnees_entreprises.to_excel(file_path, index=False)